In [1]:
!apt-get -qq update >/dev/null && apt-get -qq upgrade >/dev/null
!apt-get -qq install default-jdk openssh-client openssh-server >/dev/null
%set_env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64
import os
os.environ['PATH'] = ':'.join((os.environ['JAVA_HOME'] + '/bin', os.environ['PATH']))

Extracting templates from packages: 100%
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


In [2]:
!service ssh stop
%rm -r /root/.ssh
%mkdir /root/.ssh
!ssh-keygen -q -t rsa -P '' -f /root/.ssh/id_rsa
%cat /root/.ssh/id_rsa.pub >> /root/.ssh/authorized_keys
!chmod 0600 /root/.ssh/authorized_keys
!sed -i 's/#   StrictHostKeyChecking ask/    StrictHostKeyChecking no/' /etc/ssh/ssh_config
!service ssh start

 * Stopping OpenBSD Secure Shell server sshd
   ...done.
rm: cannot remove '/root/.ssh': No such file or directory
 * Starting OpenBSD Secure Shell server sshd
   ...done.


In [3]:
!curl -fsSL http://apache.mirrors.tds.net/hadoop/common/hadoop-3.0.0/hadoop-3.0.0.tar.gz -o ./hadoop-3.0.0.tar.gz
!tar -xzf ./hadoop-3.0.0.tar.gz
%set_env HADOOP_HOME /content/hadoop-3.0.0
%set_env HDFS_NAMENODE_USER root
%set_env HDFS_DATANODE_USER root
%set_env HDFS_SECONDARYNAMENODE_USER root
%set_env YARN_RESOURCEMANAGER_USER root
%set_env YARN_NODEMANAGER_USER root
os.environ['PATH'] = ':'.join((os.environ['HADOOP_HOME'] + '/bin', os.environ['PATH']))

curl: (22) The requested URL returned error: 404 Not Found
tar (child): ./hadoop-3.0.0.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
env: HADOOP_HOME=/content/hadoop-3.0.0
env: HDFS_NAMENODE_USER=root
env: HDFS_DATANODE_USER=root
env: HDFS_SECONDARYNAMENODE_USER=root
env: YARN_RESOURCEMANAGER_USER=root
env: YARN_NODEMANAGER_USER=root


In [4]:
!sed -i -e "s,# export JAVA_HOME=,export JAVA_HOME=$JAVA_HOME,"\
        -e "s,# export HADOOP_HOME=,export HADOOP_HOME=$HADOOP_HOME," hadoop-3.0.0/etc/hadoop/hadoop-env.sh

sed: can't read hadoop-3.0.0/etc/hadoop/hadoop-env.sh: No such file or directory


In [5]:
!echo "<configuration><property><name>fs.defaultFS</name><value>hdfs://$(hostname):8020</value></property></configuration>" >./hadoop-3.0.0/etc/hadoop/core-site.xml

/bin/sh: 1: cannot create ./hadoop-3.0.0/etc/hadoop/core-site.xml: Directory nonexistent


In [6]:
%%file ./hadoop-3.0.0/etc/hadoop/hdfs-site.xml
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>

Writing ./hadoop-3.0.0/etc/hadoop/hdfs-site.xml


IOError: ignored

In [7]:
%%file ./hadoop-3.0.0/etc/hadoop/mapred-site.xml
<configuration>
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>
</configuration>

Writing ./hadoop-3.0.0/etc/hadoop/mapred-site.xml


IOError: ignored

In [8]:
%%file ./hadoop-3.0.0/etc/hadoop/yarn-site.xml
<configuration>
    <property>
        <name>yarn.nodemanager.aux-services</name>
        <value>mapreduce_shuffle</value>
    </property>
    <property>
        <name>yarn.nodemanager.env-whitelist</name>
        <value>JAVA_HOME,HADOOP_COMMON_HOME,HADOOP_HDFS_HOME,HADOOP_CONF_DIR,CLASSPATH_PREPEND_DISTCACHE,HADOOP_YARN_HOME,HADOOP_MAPRED_HOME</value>
    </property>
</configuration>

Writing ./hadoop-3.0.0/etc/hadoop/yarn-site.xml


IOError: ignored

In [9]:
%rm -r /tmp/hadoop-root/*
!hdfs namenode -format -force -nonInteractive
!./hadoop-3.0.0/sbin/start-dfs.sh
!hdfs dfs -mkdir /user
!hdfs dfs -mkdir /user/root
!./hadoop-3.0.0/sbin/start-yarn.sh

rm: cannot remove '/tmp/hadoop-root/*': No such file or directory
/bin/sh: 1: hdfs: not found
/bin/sh: 1: ./hadoop-3.0.0/sbin/start-dfs.sh: not found
/bin/sh: 1: hdfs: not found
/bin/sh: 1: hdfs: not found
/bin/sh: 1: ./hadoop-3.0.0/sbin/start-yarn.sh: not found


In [10]:
!pip install pyspark

    99% |████████████████████████████████| 211.9MB 7.3MB/s eta 0:00:01tcmalloc: large alloc 1130004480 bytes == 0x557d072a0000 @  0x7f43cb97b107 0x557cbe8584ed 0x557cbe8dbac0 0x557cbe900d29 0x557cbe85f90e 0x557cbe924021 0x557cbe87677a 0x557cbe87b462 0x557cbe873b3a 0x557cbe87b82e 0x557cbe873b3a 0x557cbe88f809 0x557cbe8a856e 0x557cbe85f90e 0x557cbe8f0b76 0x557cbe87b650 0x557cbe87b462 0x557cbe873b3a 0x557cbe87b82e 0x557cbe873b3a 0x557cbe87b82e 0x557cbe88ed68 0x557cbe876c3f 0x557cbe88ed68 0x557cbe876c3f 0x557cbe88ed68 0x557cbe876c3f 0x557cbe88ed68 0x557cbe876c3f 0x557cbe873b3a 0x557cbe87be1f
    100% |████████████████████████████████| 211.9MB 5.3kB/s 
    100% |████████████████████████████████| 194kB 5.3MB/s 
  Running setup.py bdist_wheel for pyspark ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

In [0]:
import pandas as pd

from pyspark import SparkConf, SparkContext, HiveContext
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1bDRfC70G-6bOgzkMw8R6UfrQN8sEUlX8'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile("train.csv")

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1HQA-ljCRNduUiJQ4sZ0-rEIA-dBlsp7q'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile("test.csv")

In [17]:
conf = SparkConf().setAppName("Cloudera Project")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

ValueError: ignored

In [0]:
dftrain=pd.read_csv("train.csv")
train=hc.createDataFrame(dftrain)

In [0]:
dftest=pd.read_csv("test.csv")
test=hc.createDataFrame(dftest)

In [0]:
classes = [i for i in train.columns if i not in ["id", "comment_text"]]

In [20]:
print(classes)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [0]:
tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")
wordsData = tokenizer.transform(train)

In [0]:
hashingTF = HashingTF(inputCol="words", outputCol="hashdata")
tf = hashingTF.transform(wordsData)

In [23]:
tf.select('hashdata').take(2)

[Row(hashdata=SparseVector(262144, {19208: 1.0, 23032: 1.0, 24417: 1.0, 25000: 1.0, 29945: 1.0, 32241: 1.0, 32976: 1.0, 37852: 1.0, 46075: 1.0, 59853: 1.0, 72125: 1.0, 77971: 1.0, 81631: 1.0, 82999: 1.0, 83922: 1.0, 91677: 1.0, 97171: 1.0, 100258: 1.0, 101169: 1.0, 103838: 3.0, 110427: 1.0, 113031: 1.0, 113418: 1.0, 135568: 1.0, 139533: 1.0, 140784: 1.0, 145284: 1.0, 151536: 1.0, 164148: 1.0, 169364: 1.0, 176964: 1.0, 182267: 1.0, 192137: 1.0, 193131: 1.0, 229137: 1.0, 230921: 1.0, 231630: 1.0, 244466: 1.0, 246621: 1.0, 249835: 1.0, 253170: 1.0})),
 Row(hashdata=SparseVector(262144, {17429: 1.0, 38728: 1.0, 83815: 1.0, 88337: 1.0, 101527: 1.0, 101833: 1.0, 108541: 1.0, 125765: 1.0, 141219: 1.0, 151980: 1.0, 169364: 1.0, 169800: 1.0, 203235: 1.0, 208090: 1.0, 219140: 1.0, 242101: 1.0, 248135: 1.0, 249180: 1.0}))]

In [0]:
idf = IDF(inputCol="hashdata", outputCol="features")
idfModel = idf.fit(tf) 
tfidf = idfModel.transform(tf)

In [25]:
tfidf.select("features").first()

Row(features=SparseVector(262144, {19208: 2.244, 23032: 5.0123, 24417: 0.7386, 25000: 5.6813, 29945: 3.0517, 32241: 8.3967, 32976: 5.0285, 37852: 1.7539, 46075: 6.9564, 59853: 3.1523, 72125: 2.2744, 77971: 7.6108, 81631: 3.4198, 82999: 7.7906, 83922: 6.4709, 91677: 0.6965, 97171: 2.0163, 100258: 1.1947, 101169: 1.734, 103838: 1.2127, 110427: 2.1173, 113031: 8.9845, 113418: 2.2023, 135568: 3.5864, 139533: 2.5136, 140784: 3.0483, 145284: 7.6628, 151536: 2.2412, 164148: 6.0064, 169364: 2.4772, 176964: 1.7656, 182267: 8.648, 192137: 3.1018, 193131: 5.6703, 229137: 4.5705, 230921: 2.0429, 231630: 8.2914, 244466: 3.351, 246621: 10.0343, 249835: 6.827, 253170: 2.7021}))

In [0]:
test_tokens = tokenizer.transform(test)
test_tf = hashingTF.transform(test_tokens)
test_tfidf = idfModel.transform(test_tf)

In [0]:
test_res = test.select('id')

In [0]:
REG = 0.1

In [0]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [30]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",labelCol='toxic',featuresCol="features")
model = nb.fit(tfidf)
predictions = model.transform(test_tfidf)
predictions.show()
predictions.select("id", "probability", "prediction").show(20)

+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|              id|        comment_text|               words|            hashdata|            features|       rawPrediction|         probability|prediction|
+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|00001cee341fdb12|Yo bitch Ja Rule ...|[yo, bitch, ja, r...|(262144,[7597,157...|(262144,[7597,157...|[-3557.1007487253...|[5.38119059830858...|       1.0|
|0000247867823ef7|== From RfC == 

...|[==, from, rfc, =...|(262144,[1950,158...|(262144,[1950,158...|[-377.33361105852...|[1.0,1.5326354255...|       0.0|
|00013b17ad220c46|" 

 == Sources =...|[", , , , ==, sou...|(262144,[1950,351...|(262144,[1950,351...|[-721.63859105012...|[1.0,4.1663490590...|       0.0|
|00017563c3f7919a|:If you have a lo...|[:if, you, have, ...|(262

In [0]:
extract_prob = F.udf(lambda x: float(x[1]), T.FloatType())


In [0]:
test_probs = []
for col in classes:
    print(col)
    nb = NaiveBayes(smoothing=1.0, modelType="multinomial",labelCol=col,featuresCol="features")
    nbModel = nb.fit(tfidf)
    #print("...predicting")
    res = nbModel.transform(test_tfidf)
    #print("...appending result")
    test_res = test_res.join(res.select('id', 'probability'), on="id")
    print("...extracting probability")
    test_res = test_res.withColumn(col, extract_prob('probability')).drop("probability")
    test_res.show()

toxic
...extracting probability
+----------------+-------------+
|              id|        toxic|
+----------------+-------------+
|000968ce11f5ee34| 8.949112E-12|
|00491682330fdd1d|          0.0|
|008eb47c4684d190|          0.0|
|00d251f47486b6d2|8.2412777E-10|
|0114ae82c53101a9|          1.0|
|012c7429c5a34290| 6.189828E-11|
|015017ec394a264e|   0.99994123|
|01d94c94a86a4327|          0.0|
|020eb3a1af28453f|          1.0|
|0216909e11cfeac0|          0.0|
|026460a698a91698|          0.0|
|027cc0ae6a33392e| 1.0237167E-6|
|02a5d713614fad26| 3.4725966E-7|
|02aabe84e138c05f| 3.268762E-12|
|02f2b8d194b06506| 2.1854843E-5|
|02f60289932b3234|  1.105516E-5|
|0311a466e20edcb4|          0.0|
|034ea2a7c86e7e49|          0.0|
|039acb76708e55a6|    8.207E-42|
|03c81594154c4651|          0.0|
+----------------+-------------+
only showing top 20 rows

severe_toxic
...extracting probability
+----------------+-------------+-------------+
|              id|        toxic| severe_toxic|
+----------------

In [0]:
test_res_pan = test_res.toPandas()

In [0]:
test_res_pan.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,000968ce11f5ee34,8.949112e-12,2.923570e-37,1.509294e-10,2.178481e-33,1.121275e-13,9.224082e-33
1,000bafe2080bba82,1.381710e-27,0.000000e+00,6.047047e-31,0.000000e+00,2.048829e-30,1.434845e-34
2,0009734200a85047,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0003806b11932181,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,000bf0a9894b2807,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,00091c35fa9d0465,1.121513e-10,0.000000e+00,2.217137e-09,0.000000e+00,4.744609e-07,8.612464e-09
6,00025358d4737918,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,000834769115370c,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,0002eadc3b301559,1.039787e-10,8.407791e-45,2.898816e-13,5.430032e-42,1.029478e-14,1.249761e-27
9,00001cee341fdb12,1.000000e+00,6.789010e-24,1.000000e+00,2.873344e-29,1.000000e+00,1.000000e+00


In [0]:
test_res_pan.to_csv("results.csv", index=False)

NameError: ignored

In [0]:
4/AAD4iDh57VwLMV8CjkeDUR8PAL-QSDgie1zi11ET194MvLeSlkEXqrwfrom google.colab import files

files.download('results.csv')